In [11]:
import os
import pandas as pd
import numpy as np
from Bio import SeqIO


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [117]:
reference = '/home/laura/DATABASES/REFERENCES/COVID/NC_045512.2.fasta'
intermediate = '/home/laura/ANALYSIS/covidma/Compare/2020-11-24_covidma/covidma.revised.final.tsv'
cov_folder = '/home/laura/ANALYSIS/covidma/Stats/Coverage'

In [106]:
reference_seq.seq

Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA', SingleLetterAlphabet())

In [107]:
"""for seq_record in SeqIO.parse(reference, "fasta"):
    print(seq_record.id)
    print(repr(seq_record.seq))
    print(len(seq_record))"""

'for seq_record in SeqIO.parse(reference, "fasta"):\n    print(seq_record.id)\n    print(repr(seq_record.seq))\n    print(len(seq_record))'

In [108]:
def reference_to_list(ref_file):
    with open(ref_file, 'r') as f:
        content = f.read()
        content_list = content.split('\n')
        header = content_list[0]
        sequence = ("").join(content_list[1:])
    return list(sequence)

In [109]:
reflist = list(SeqIO.read(reference, "fasta").seq)

In [110]:
reflist[0:10]

['A', 'T', 'T', 'A', 'A', 'A', 'G', 'G', 'T', 'T']

In [141]:
def create_consensus(reference, highfreq_df, coverage_folder, out_folder):
    df = pd.read_csv(highfreq_df, sep="\t")

    for sample in df.columns[3:]:
        coverage_folder = os.path.abspath(coverage_folder)
        cov_file = os.path.join(coverage_folder, sample + ".cov")
        reflist = list(SeqIO.read(reference, "fasta").seq)
        
        dfsample = df[['Position', sample]]
        for _, row in dfsample.iterrows():
            if str(row[sample]) == '1':
                postition_list = row.Position.split("|")
                ref = postition_list[1]
                pos = int(postition_list[2])
                alt = postition_list[3]
                if reflist[pos - 1] == ref:
                    reflist[pos - 1] = alt
        covdf = pd.read_csv(cov_file, sep="\t", names=["#CHROM", "POS", "COV"])
        uncovered = covdf[covdf.COV == 0]
        for _, row in uncovered.iterrows():
            reflist[row.POS - 1] = 'N'
        output_file = os.path.join(out_folder, sample + ".consensus.fasta")
        with open(output_file, 'w+') as fout:
            fout.write('>{}\n{}\n'.format(sample,('').join(reflist)))

In [142]:
create_consensus(reference, intermediate, cov_folder, '/home/laura/ANALYSIS/covidma/Consensus')

20091271
20179107
20089101
20069302
